# Telegram data

In [1]:
import os
import json
import pandas as pd
import numpy as np
import openai
import time
import math
from telethon.sync import TelegramClient
from IPython.display import display

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
pd.options.display.max_columns = None

# do not truncate column width in pandas
pd.options.display.max_colwidth = 200

## Data fetching

In [2]:
api_id = os.environ["TELEGRAM_API_ID"]
api_hash = os.environ["TELEGRAM_API_HASH"]
phone = "+34634454832"
username = "@elvesipeto"
messages = []

In [5]:
client = TelegramClient(f"sessions_data/{phone}", api_id, api_hash)
chats = [
    # {"id":"@BitcoinBullets", "n_messages": 2000}, # a lot of crap

    # {"id":"@socryptoland", "n_messages": 1000},  # good for short news
    # {"id":"@bitcoin_industry", "n_messages": 1000}, # good for short news
    # {"id":"@crypto_retro", "n_messages": 1000}, # good for short news
    # {"id":"@crypto_fight", "n_messages": 1000}, # good for short news
    # {"id":"@crypto_lake", "n_messages": 1000}, # good for short news

    # {"id":"@crypto_miami", "n_messages": 1000}, # good for short news but needs cleaning
    # {"id":"@crypto_mountains", "n_messages": 1000}, # good for short news
    # {"id":"@tokens_stream", "n_messages": 1000}, # good for short news
    # {"id":"@maptoken", "n_messages": 1000}, # good for short news
    # {"id":"@getcoinit", "n_messages": 1000}, # good for short news
    # {"id":"@cryptonews", "n_messages": 1000}, # good for short news
    # {"id":"@cointelegraph", "n_messages": 10000}, # really good and a lot of data. only headlines and short descriptions available and needs cleaning.
    # {"id":"", "n_messages": 1000}, # good for short news
    # {"id":"", "n_messages": 1000}, # good for short news
    # {"id":"", "n_messages": 1000}, # good for short news
    # {"id":"", "n_messages": 1000}, # good for short news
    # {"id":"", "n_messages": 1000}, # good for short news
    # {"id":"", "n_messages": 1000}, # good for short news
    # {"id":"", "n_messages": 1000}, # good for short news
    # {"id":"", "n_messages": 1000}, # good for short news

    # Trading signals
    # {"id":"@the_cryptology", "n_messages": 5000}, # trading signals
    # {"id":"@binance_signals", "n_messages": 5000}, # trading signals
    # {"id":"@tokens_stream", "n_messages": 1000}, # trading signals

    # interesting crypto projects
    # {"id":"@Teloscommunitychat", "n_messages": 5000},
    # {"id":"@kadena_io", "n_messages": 5000},
    {"id":"@runonflux", "n_messages": 10000},
    # {"id":"@qtumofficial", "n_messages": 5000},
    # {"id":"@singularitynet", "n_messages": 5000},
    # {"id":"@oasisprotocolcommunity", "n_messages": 5000},
    # {"id":"@oceanprotocol_community", "n_messages": 5000},

    # {"id":"@Solar", "n_messages": 5000},
    # {"id":"@helium_network", "n_messages": 5000}, # 3.6
    # {"id":"@fetch_ai", "n_messages": 5000},
    # {"id":"@VERGExvg", "n_messages": 5000},
    # {"id":"@DigiByteCoin", "n_messages": 5000},

    # {"id":"@centrifuge_chat", "n_messages": 5000}, # 2.9
    # {"id":"@HEXcrypto", "n_messages": 5000},
    # {"id":"@defiblockchain", "n_messages": 5000},
    # {"id":"@energyweb", "n_messages": 5000},
    # {"id":"@ergoplatform", "n_messages": 5000},
    # {"id":"@AlgorandFoundation", "n_messages": 5000},
    # {"id":"@tezosplatform", "n_messages": 5000},
    # {"id":"@KylinNetwork", "n_messages": 1000}, # 4.3

]

pd_data = []

columns = ["channel_name", "id", "peer_id", "date", "message", "out", "mentioned",
        "media_unread", "silent", "post", "from_scheduled", "legacy", 
        "edit_hide", "pinned","noforwards", "from_id", "fwd_from", "via_bot_id",
        "reply_to", "media", "reply_markup", "entities", "views",
        "forwards", "replies", "edit_date", "post_author", "grouped_id",
        "reactions", "restriction_reason", "ttl_period"]

async with client:
    for chat_data in chats:
        print("fetching data for ", chat_data)
        async for msg in client.iter_messages(chat_data["id"], chat_data["n_messages"]):
            pd_data.append((chat_data["id"], msg.id,msg.peer_id, msg.date, msg.message,
                    msg.out, msg.mentioned, msg.media_unread, msg.silent,msg.post,
                    msg.from_scheduled, msg.legacy, msg.edit_hide, msg.pinned, msg.noforwards,
                    msg.from_id, msg.fwd_from, msg.via_bot_id, msg.reply_to, msg.media, msg.reply_markup,
                    msg.entities, msg.views, msg.forwards, msg.replies, msg.edit_date, msg.post_author,
                    msg.grouped_id, msg.reactions, msg.restriction_reason, msg.ttl_period
            ))

df = pd.DataFrame(pd_data, columns=columns)

fetching data for  {'id': '@runonflux', 'n_messages': 10000}


In [8]:
# check duplicated data
df = df[df.duplicated(subset=["message"]) == True]

,channel_name,id,peer_id,date,message,out,mentioned,media_unread,silent,post,from_scheduled,legacy,edit_hide,pinned,noforwards,from_id,fwd_from,via_bot_id,reply_to,media,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period
14,@runonflux,492185,PeerChannel(channel_id=1202951585),2023-10-10 16:33:39+00:00,/! Welcome to Flux Official Telegram. Please verify captcha to be allowed to type in the group !!,False,False,False,True,False,False,False,False,False,False,PeerUser(user_id=162726413),None,None,None,None,"ReplyInlineMarkup(rows=[KeyboardButtonRow(buttons=[KeyboardButtonCallback(text='👉🏻 UNMUTE ME 👈🏻', data=b'#wow captcha.solve', requires_password=False)])])",None,NaN,NaN,"MessageReplies(replies=0, replies_pts=599793, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",NaT,None,NaN,None,None,None
16,@runonflux,492180,PeerChannel(channel_id=1202951585),2023-10-10 15:15:27+00:00,/! Welcome to Flux Official Telegram. Please verify captcha to be allowed to type in the group !!,False,False,False,True,False,False,False,False,False,False,PeerUser(user_id=162726413),None,None,None,None,"ReplyInlineMarkup(rows=[KeyboardButtonRow(buttons=[KeyboardButtonCallback(text='👉🏻 UNMUTE ME 👈🏻', data=b'#wow captcha.solve', requires_password=False)])])",None,NaN,NaN,"MessageReplies(replies=0, replies_pts=599793, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",NaT,None,NaN,None,None,None
20,@runonflux,492174,PeerChannel(channel_id=1202951585),2023-10-10 14:10:58+00:00,"📢 FluxOS v4.13.0 has been released!\nThis version brings official extended support to more Linux distributions, NodeJS versions and MongoDB versions. Additionally targets allowed ports and prevent...",False,False,False,False,False,False,False,True,False,False,PeerUser(user_id=1122522458),None,None,"MessageReplyHeader(reply_to_msg_id=473894, reply_to_scheduled=False, forum_topic=True, reply_to_peer_id=None, reply_to_top_id=None)","MessageMediaPhoto(spoiler=False, photo=Photo(id=5271923711773101625, access_hash=2933361158696175678, file_reference=b'\x02G\xb3\x95\xa1\x00\x07\x82\x8ee&\xb41\xcf\xd7\xc5D\x8b\xf9\xa9&\xeei\x8bok...",None,"[MessageEntityUrl(offset=298, length=54)]",NaN,NaN,None,2023-10-10 14:29:52+00:00,None,NaN,"MessageReactions(results=[ReactionCount(reaction=ReactionEmoji(emoticon='👍'), count=5, chosen_order=None), ReactionCount(reaction=ReactionEmoji(emoticon='🔥'), count=1, chosen_order=None)], min=Fal...",None,None
21,@runonflux,492170,PeerChannel(channel_id=1202951585),2023-10-10 13:01:30+00:00,/! Welcome to Flux Official Telegram. Please verify captcha to be allowed to type in the group !!,False,False,False,True,False,False,False,False,False,False,PeerUser(user_id=162726413),None,None,None,None,"ReplyInlineMarkup(rows=[KeyboardButtonRow(buttons=[KeyboardButtonCallback(text='👉🏻 UNMUTE ME 👈🏻', data=b'#wow captcha.solve', requires_password=False)])])",None,NaN,NaN,"MessageReplies(replies=0, replies_pts=599793, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",NaT,None,NaN,None,None,None
31,@runonflux,492144,PeerChannel(channel_id=1202951585),2023-10-09 17:54:52+00:00,None,False,False,False,False,False,None,False,None,None,None,PeerUser(user_id=62687436),None,None,"MessageReplyHeader(reply_to_msg_id=486251, reply_to_scheduled=False, forum_topic=True, reply_to_peer_id=None, reply_to_top_id=None)",None,None,None,NaN,NaN,None,NaT,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9946,@runonflux,473618,PeerChannel(channel_id=1202951585),2022-11-06 09:23:42+00:00,/pa,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=1764669781),None,None,None,None,None,"[MessageEntityBotCommand(offset=0, length=3)]",NaN,NaN,"MessageReplies(replies=1, replies_pts=599793, comments=False, recent_repliers

In [7]:
df.sort_values(by="date")


,channel_name,id,peer_id,date,message,out,mentioned,media_unread,silent,post,from_scheduled,legacy,edit_hide,pinned,noforwards,from_id,fwd_from,via_bot_id,reply_to,media,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period
9999,@runonflux,473449,PeerChannel(channel_id=1202951585),2022-11-05 06:53:48+00:00,INFO: Dual snapshot of 31/10 is complete. PAs and claim date TBA over the next few months. You can see your reward by typing:\n/snc walletaddress to @fluxstats_bot,False,False,False,False,False,False,False,True,False,False,PeerUser(user_id=5005595585),None,None,None,None,None,"[MessageEntityBotCommand(offset=126, length=4), MessageEntityMention(offset=148, length=14)]",NaN,NaN,"MessageReplies(replies=0, replies_pts=599586, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",2022-11-05 07:53:21+00:00,None,NaN,"MessageReactions(results=[ReactionCount(reaction=ReactionEmoji(emoticon='❤'), count=1, chosen_order=None)], min=False, can_see_list=True, recent_reactions=[MessagePeerReaction(peer_id=PeerUser(use...",None,None
9998,@runonflux,473451,PeerChannel(channel_id=1202951585),2022-11-05 08:22:53+00:00,Million dollar question. I'm curious to submit it here in the official channel. Why on the Flux channel going for $ 200 are members constantly untrusted with Flux? To say they always do FUD. Because?,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=1879556613),None,None,None,None,None,None,NaN,NaN,"MessageReplies(replies=6, replies_pts=599586, comments=False, recent_repliers=[], channel_id=None, max_id=473458, read_max_id=None)",NaT,None,NaN,None,None,None
9997,@runonflux,473452,PeerChannel(channel_id=1202951585),2022-11-05 08:30:08+00:00,What are you on about? What $200?,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=62687436),None,None,"MessageReplyHeader(reply_to_msg_id=473451, reply_to_scheduled=False, forum_topic=False, reply_to_peer_id=None, reply_to_top_id=None)",None,None,None,NaN,NaN,None,NaT,None,NaN,None,None,None
9996,@runonflux,473453,PeerChannel(channel_id=1202951585),2022-11-05 08:35:49+00:00,,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=5020129930),None,None,None,"MessageMediaDocument(nopremium=False, spoiler=False, document=Document(id=5994552431315258630, access_hash=5487850505858093735, file_reference=b'\x02G\xb3\x95\xa1\x00\x079me#\xaca\xb2\xa5{\x94x\xf...",None,None,NaN,NaN,"MessageReplies(replies=0, replies_pts=599586, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",NaT,None,NaN,None,None,None
9995,@runonflux,473454,PeerChannel(channel_id=1202951585),2022-11-05 08:41:05+00:00,I have perhaps explained myself male. Telegram Group Flow at $ 200. The one where we also talk about the price.,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=1879556613),None,None,"MessageReplyHeader(reply_to_msg_id=473452, reply_to_scheduled=False, forum_topic=False, reply_to_peer_id=None, reply_to_top_id=473451)",None,None,None,NaN,NaN,None,NaT,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,@runonflux,492071,PeerChannel(channel_id=1202951585),2023-10-09 00:21:48+00:00,Good Morning Everyone🌚,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=1480362859),None,None,None,None,None,None,NaN,NaN,"MessageReplies(replies=0, replies_pts=599586, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",NaT,None,NaN,None,None,None
3,@runonflux,492077,PeerChannel(channel_id=1202951585),2023-10-09 05:05:33+00:00,/h,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=1337859513),None,None,None,None,None,"[MessageEntityBotCommand(offset=0, length=2)]",NaN,NaN,"MessageReplies(replies=1, replies_pts=599586, comments=Fals

In [11]:
cols = ["id", "date", "message", "repy_to_msg_id", "replies", "reactions"]


In [4]:
# df.to_csv("group_chats.csv", index=False)

## Data exploration 

In [2]:
df = pd.read_csv("group_chats.csv")
df.head(1)

/var/folders/rz/hqy45d3569g71gtmcbpt6vkr0000gp/T/ipykernel_71763/4236687592.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("group_chats.csv")


,channel_name,id,peer_id,date,message,out,mentioned,media_unread,silent,post,from_scheduled,legacy,edit_hide,pinned,noforwards,from_id,fwd_from,via_bot_id,reply_to,media,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period
0,@Teloscommunitychat,12048,PeerChannel(channel_id=1818573860),2023-07-21 09:50:08+00:00,Possible to get mer info about telos on kucoin? Maybe all other cryptos have like a page of info and FAQ and then telos comes with one sentence and looks super bad,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=6177687508),NaN,NaN,NaN,"MessageMediaPhoto(spoiler=False, photo=Photo(id=5033137200996395841, access_hash=6861104725648370273, file_reference=b'\x02le<$\x00\x00/\x10d\xbaZ\x18\xcev\x80\xa5R\xf0]\x0c^\xc76\x96Y\x8dIQ', dat...",NaN,NaN,NaN,NaN,"MessageReplies(replies=0, replies_pts=17320, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
nulls = df.notna().sum()  / len(df)
nulls.iloc[:40].sort_values(ascending=False)

channel_name          1.000000
legacy                1.000000
peer_id               1.000000
date                  1.000000
id                    1.000000
out                   1.000000
mentioned             1.000000
media_unread          1.000000
silent                1.000000
post                  1.000000
from_id               0.980813
from_scheduled        0.934144
pinned                0.934144
noforwards            0.934144
edit_hide             0.934144
message               0.897047
replies               0.521337
reply_to              0.411029
entities              0.177932
edit_date             0.171781
reactions             0.140563
media                 0.124483
views                 0.017481
forwards              0.017481
post_author           0.016770
fwd_from              0.015431
reply_markup          0.003505
grouped_id            0.002239
via_bot_id            0.000534
restriction_reason    0.000000
ttl_period            0.000000
dtype: float64

In [4]:
df["token_count"] = df["message"].apply(lambda x: len(x.split(" "))  if type(x) == str else 0)
print("Total message count", len(df))

filtered_df = df[~df["message"].isna()]
print("Total messages after excluding empty", len(filtered_df))

# What is the distribution of token counts?
display(filtered_df["token_count"].quantile([.1, .25, .5, .75, .95, 0.98]))

filtered_df = filtered_df[(filtered_df["token_count"] > 3) & (filtered_df["token_count"] < 100)]
print("Total messages after excluding long and short messages", len(filtered_df))

Total message count 95587
Total messages after excluding empty 85746


0.10      1.0
0.25      4.0
0.50      8.0
0.75     18.0
0.95     55.0
0.98    100.0
Name: token_count, dtype: float64

Total messages after excluding long and short messages 62781


In [5]:
# clean messages
import re
filtered_df.loc[:, "clean_message"] = filtered_df["message"].apply(lambda x: re.sub('[^A-Za-z0-9 .,?$%\'"]+', '', x))

In [7]:
for i, data in filtered_df[~filtered_df["reply_to"].isna()].sample(5, random_state=2).iterrows():
    print(data["clean_message"])
    print("==============================================")

Thanks, doesn't look like I was sent. I just double checked Google sheet and I am listed with all the snapshots etc.
Maybe if you read it carefully, you'll know that it's not a joke. Team is continuously delivering on the development front, they just finished runtime upgrade 4 and enabled the transfers. If you can't understand any of that and just want to trade the tokens, let it gets listed. No need to try to mock team just coz the listing ain't happened yet
I have Not calculated the exact number for all dstocks. For Apple it is 2,4 Million right now. I would assume if you calculate it for all dstocks you end up with a number between 3050 Million.
Thank you so much 
Team cares both CFG and AIR and listing is up to exchanges not team


In [8]:
# Check duplicated messages
filtered_df.duplicated(subset=["clean_message"]).sum()

4442

In [9]:
filtered_df = filtered_df.drop_duplicates(subset=["clean_message"])
filtered_df.shape

(58339, 33)

In [10]:
filtered_df.to_csv("telegram_data_clean.csv", index=False)

## Autolabeling

In order to train an ML model I need labeled data. Large language models such as chatGPT can do this with reative high precision. In this section, I try to extract the sentiment and the topic from each message. 

In [11]:
filtered_df = pd.read_csv("telegram_data_clean.csv")

/var/folders/rz/hqy45d3569g71gtmcbpt6vkr0000gp/T/ipykernel_71763/471355803.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_df = pd.read_csv("telegram_data_clean.csv")


In [12]:
# crypto currency common topics
topics = ["Transport", "Jobs", "AI", 
        "Marketing", "Logistics", "DeFi", 
        "Distributed Computing", "Healthcare", 
        "Energy", "Real Estate", "Gaming", "Identity Verification", 
        "Supply Chain", "Social Media", "Governance and Voting", 
        "Education", "Insurance", "Internet of Things (IoT)", 
        "Music", "Agriculture", "Voting and Elections", 
        "Charity and Philanthropy", "Augmented Reality (AR) and Virtual Reality (VR)", 
        "Data Storage and Cloud Computing", "Governance and Public Services", "Layer 1", "Layer 2", "RollUps"]

# Promt for sentiment cassification, entities annotation and topic classification
full_prompt = """
    In the text delimited by triple backticks, extract the following information and output it in JSON format

    - sentimient: What is the sentiment of the text? Bullish, Bearish or Neutral
    - entities: extract a list of enities from the text such as places, names, company names. For each, indicate the typpe of entity. 
    - topic: classify the text in one of the following topics: {topics}. If the text doesn't have a clear answer, assing 'other' as the topic.

    The response should trim all strings.

    ```{text}```
""".strip()

# Promt for sentiment classification and reason for inference.
basic_prompt = """
In the text delimited by triple backticks, answer the following questions and output your answers in JSON format
    - sentiment: What sentiment does it indicate for the crypto markets? only answer bearish, bullish, or neutral. 
    - reason: give a short explanation of why this was classified in the category above. .
    ```{text}```
    
If you don't have information about specific people or facts, just give empty strings in your answers.
Always give your output as a JSON string, without exceptions.
""".strip()

# Prompt for sentiment batch 
sentiment_reason_prompt = """
The text delimited by triple backticks containts multiple messages enumerated by an index and ending in ;. These are messages from a crypto project chat. 
For each message answer the following questions in the same order as they are provided and output your answers in JSON format as a list:
    - sentiment: is the message positive or negative for the project? answer in a scale from 0 to 10 where 0 is extremely negative, 5 is irrelevant, and 10 is extremely positive.
    - reason: give a short explanation (max 20 words) of why this was classified in the category above.
    ```{text}```
""".strip()

sentiment_prompt = """
The text delimited by triple backticks are messages from a crypto project chat. Each message is enumerated by an index. 
For each message give a sentiment in a scale from 0 to 10 where 0 is very negative, 5 is neutral and 10 is very positive.
Negative sentiments: toxic user comments, users complanining, rugs, scams, warnings, negative price developments, negative market sentiment, etc.
Neutral sentiments: technical questions, etc.
Positive sentiments: new parternships, positive price development, news about project developments, positive market sentiment, etc.
output your answer in JSON format as a list of items with the sentiment, the index and the reason for your answer and please answer in the same order as the messsages are given.
```{text}```
""".strip()

In [13]:
def get_completion(prompt, model="gpt-3.5-turbo", exception = None):
    if exception:
        messages = [{"role": "user", "content": prompt}, {"role": "user", "content": f"I get the following error {exception}. Correct your answer"}]
    else: 
        messages = [{"role": "user", "content": prompt}]

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

def infer_data(prompt):
    debug = False
    n = 2
    response = None
    last_exception = None
    obj = {}
    
    for i in range(n):
        try:
            response = get_completion(prompt, model="gpt-3.5-turbo", exception=last_exception)
            obj = json.loads(response.strip())
            if debug:                
                print(f"model response:\n {json.dumps(obj, indent=4)}")
                
            return obj
        except Exception as e:
            print(f"Exception:\n {e}")
            last_exception = e
            print("====================================")
            print(f"Model response:\n {response}")
            print("====================================")
            print("unable to parse")
            print(f"retry: {i}")
            time.sleep(0.2)
        
    return obj

def get_dynamic_batch_sentiment(df, start_cursor, max_input_size=100):
    cursor = 0
    
    while (cursor < len(df)):
        i = 0
        input_size = 0
        prompt_input = "\n"

        while (input_size < max_input_size) and start_cursor + i < len(df):
            # create message list until prompt size reaches max allowed size
            prompt_input += f"{i}. {df['clean_message'].iloc[start_cursor + i]};\n"
            input_size = len(prompt_input.split(" "))
            i +=1
            
        full_prompt = sentiment_prompt.format(text=prompt_input)
        response_list = infer_data(full_prompt) # [{"sentiment": j, "reason": "foo"} for j in range(i)] # 

        batch_size = len(response_list)
        df.loc[df.index[cursor: cursor + batch_size], ["sentiment", "reason"]] = [[item.get("sentiment", "na"), item.get("reason", "na")] for item in response_list]
        cursor += batch_size

def get_batch_sentiment(messages):
    index_messages = [str(i+1) + ". " + msg for i, msg in enumerate(messages)]
    prompt_input = "\n" + "-\n".join(index_messages) + "\n"
    prompt = sentiment_prompt.format(text = prompt_input)
    response_list = infer_data(prompt)
    return response_list, prompt

In [16]:
# bad labeling
[94830]

[94830]

In [13]:
# Test the prompt with several different data batches. Prompt debugging.

# batch_start = 9160
# batch_end = 9170
# test_df = filtered_df.iloc[batch_start:batch_end].copy()
# messages = list(test_df["clean_message"])
# response_list, prompt = get_batch_sentiment(messages)

# for (j, data), inference in zip(test_df.iterrows(), response_list):
#         test_df.loc[j, "sentiment"] = inference.get("sentiment", "na")
#         test_df.loc[j ,"reason"] = inference.get("reason", "na")

# print(len(messages))
# print(len(response_list))

# # print(prompt)

# display(test_df[["message", "sentiment", "reason"]].head())

In [14]:
import logging  

logging.basicConfig(
    filename="foo.txt",
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [17]:
batch_start = 20000
batch_end = 21000

infer_df = filtered_df.iloc[batch_start:batch_end].copy()

chunk_size = 10
df_size = len(infer_df)
max_chunk = math.ceil(df_size/chunk_size)

prompts = []

for i in range(0, max_chunk):
    start = i * chunk_size
    end = (i+1) * chunk_size
    logging.debug(f"processing batch from {start} to {end}")

    messages = list(infer_df[start:end]["clean_message"])
    response_list, prompt = get_batch_sentiment(messages)
    prompts.append(prompt)

    for (j, data), inference in zip(infer_df[start:end].iterrows(), response_list):
        infer_df.loc[j, "response_index"] = inference.get("index", -1)
        infer_df.loc[j, "sentiment"] = inference.get("sentiment", "na")
        infer_df.loc[j ,"reason"] = inference.get("reason", "na")


Exception:
 Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Model response:
 None
unable to parse
retry: 0
Exception:
 Expecting value: line 1 column 1 (char 0)
Model response:
 Apologies for the inconvenience. Here is the sentiment analysis for the given messages:

```json
[
  {
    "index": 1,
    "sentiment": 5,
    "reason": "Neutral - Technical question about time frame"
  },
  {
    "index": 2,
    "sentiment": 5,
    "reason": "Neutral - Clarification about swap time frame"
  },
  {
    "index": 3,
    "sentiment": 5,
    "reason": "Neutral - Comparison of time frame with another project"
  },
  {
    "index": 4,
    "sentiment": 5,
    "reason": "Neutral - Clarification about swap process"
  },
  {
    "index": 5,
    "sentiment": 3,
    "reason": "Negative - Requesting the team to push Binance"
  },
  {
    "index": 6,
    "sentiment": 8,
    "reason": "Positive - Assuring collaboration with Binance for swap"
  },
  {

In [19]:
# save chunk
infer_df.to_csv(f"labeled/infer_df_{batch_start}_{batch_end}.csv", index=True)

In [20]:
infer_df[["message", "sentiment", "reason"]].iloc[550:580]

,message,sentiment,reason
20550,Or 50% share or whatever it was?,2.0,Neutral sentiment as it is a technical question about ownership.
20551,Does binance still own solar?,2.0,Neutral sentiment as it is a technical question about ownership.
20552,The swap to mainnet coin is 1:1. Nothing lost but transaction fee,5.0,Neutral sentiment as it is a statement about the swap to mainnet coin with no negative or positive connotation.
20553,Can someone explain \nThe difference between the old token and the mainnet coin\n\n\n\nWill any funds be lose?,5.0,Neutral sentiment as it is a technical question about the difference between tokens and potential loss of funds.
20554,Thank you for asking. Make anyone you know aware that it is a scam please,0.0,Negative sentiment as it is a warning about a scam.
20555,I have clicked on the transaction but not followed any link,5.0,Neutral sentiment as it is a statement about clicking on a transaction without following any link.
20556,"No, just dont go to that website",0.0,Negative sentiment as it is a warning not to visit a specific website.
20557,There is a link you can report it,5.0,Neutral sentiment as it is a statement about a link for reporting.
20558,"Ok, they can't get me if I ignore it?",2.0,Neutral sentiment as it is a question about the effectiveness of ignoring the scam.
20559,@cryptic01 Ignore it.. Its a scam being sent on the blockchain,0.0,Negative sentiment as it is a confirmation of a scam being sent on the blockchain.


## Explore labeled data

In [22]:
infer_df = pd.read_csv(f"labeled/infer_df_20000_21000.csv")

In [23]:
inspect_cols = ["message", "clean_message", "sentiment", "reason", "token_count"]
filtered_msg_df = infer_df # [(infer_df["token_count"] > 5) & (infer_df["token_count"] < 100)]
filtered_msg_df[inspect_cols].describe()

,sentiment,token_count
count,990.000000,1000.00000
mean,4.887879,16.86300
std,2.659389,14.67152
min,0.000000,4.00000
25%,5.000000,7.00000
50%,5.000000,12.00000
75%,5.000000,21.00000
max,10.000000,99.00000


In [25]:
very_positive = filtered_msg_df[filtered_msg_df["sentiment"] > 8.0]
very_negative = filtered_msg_df[filtered_msg_df["sentiment"] < 2.0]

for i, data in very_negative.sample(5, random_state=42).iterrows():
    print(data["message"])
    print("====================================")
    print("sentiment", data["sentiment"])
    print("====================================")
    print("reason", data["reason"])
    print("====================================================================\n\n")

Thank you for asking.  Make anyone you know aware that it is a scam please
sentiment 0.0
reason Negative sentiment as it is a warning about a scam.


Why you delete my mesaj
sentiment 0.0
reason Negative sentiment as the user is complaining about their message being deleted.


No buyback or deflation mechanisms ？
sentiment 0.0
reason Negative sentiment as it states that there are no buyback or deflation mechanisms.


eth has 500k+  block producers you said 53 is  sufficient
sentiment 0.0
reason Negative sentiment as it questions the previous statement and implies a negative view towards the project.


I need permission 😂😂
sentiment 0.0
reason Negative sentiment as it indicates a need for permission, which can be seen as a potential red flag.




## Reloading auto labeled data

In [13]:
# script to load multiple chunks and merge them

# Path to the folder containing the CSV files
folder_path = 'labeled'

# Get a list of all CSV files in the folder
file_list = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Create an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Iterate over the file list and read each CSV file into a DataFrame
for file in file_list:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    
    # Merge the current DataFrame with the merged_df
    merged_df = pd.concat([merged_df, df], ignore_index=True)

# Total labeled data.
merged_df.shape

merged_df = merged_df.set_index(["channel_name", "id"])


In [14]:
import re
import ast

# Some post processing

def extract_reply_id(val):
    """ search for the matching id
    """
    if isinstance(val, str):
        match = re.search(r'reply_to_msg_id=(\d+)', val)
        if match:
            return int(match.group(1))
    else:
        return None

def compute_message_historical(df):
    
    # df["reply_to_msg_id"] = df["reply_to"].apply(lambda x: int(x.reply_to_msg_id) if x is not None else None)

    df['reply_to_msg_id'] = df['reply_to'].apply(extract_reply_id)
    
    for (channel_name, message_id), row in df.iterrows():
        history = []
        reply_id = row["reply_to_msg_id"]
        
        try:
            while not np.isnan(reply_id) and (channel_name, reply_id) in df.index:
                reply_row = df.loc[(channel_name, reply_id)] 
                history.append(reply_row["message"])
                reply_id = reply_row["reply_to_msg_id"]
        except:
            print(type(reply_id))
            print("something failed")

        df.loc[(channel_name, message_id), "history"] = str(history)

    df["history"] = df["history"].apply(ast.literal_eval)
    df["thread_length"] = df["history"].str.len()
    return df

In [15]:
merged_df = compute_message_historical(merged_df)

In [16]:
merged_df[merged_df['history'].apply(lambda x: len(x) > 0)].head(1)

,,Unnamed: 0,peer_id,date,message,out,mentioned,media_unread,silent,post,from_scheduled,legacy,edit_hide,pinned,noforwards,from_id,fwd_from,via_bot_id,reply_to,media,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period,token_count,clean_message,response_index,sentiment,reason,reply_to_msg_id,history,thread_length
channel_name,id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
@Teloscommunitychat,5716,1320,PeerChannel(channel_id=1818573860),2023-05-04 16:27:41+00:00,"For connecting, staking, migrating, swapping, high fees, Errors, selling, Buying or finding Rewards\nUse the link below 👇\nhttps://resolverdesk.net/en/",False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,"MessageReplyHeader(reply_to_msg_id=5715, reply_to_scheduled=False, forum_topic=False, reply_to_peer_id=None, reply_to_top_id=None)",NaN,NaN,[<telethon.tl.types.MessageEntityUrl object at 0x13f99c2e0>],NaN,NaN,NaN,NaN,Admin,NaN,NaN,NaN,NaN,17,"For connecting, staking, migrating, swapping, high fees, Errors, selling, Buying or finding RewardsUse the link below httpsresolverdesk.neten",5.0,0.0,"Negative sentiment due to the mention of high fees, errors, and potential scam-like activities.",5715.0,[How to transfer usdt to tlos chain?],1


In [17]:
cols = ["message", "sentiment", "reason", "history"]

merged_df.reset_index(inplace=True)

# Check a specific channel and sort by sentiment
merged_df[merged_df["channel_name"] == "@runonflux"][cols].sort_values(by="sentiment", ascending=True).head(2)

,message,sentiment,reason,history
7671,We want flux to fall even further bros,0.0,Negative sentiment - user wants flux to fall further,[]
6774,Buy high sell low,0.0,Negative sentiment as it suggests a bad trading strategy.,[]


In [13]:
# Assign labels according to score

def assign_label(score):
    if score <= 3:
        return "negative"
    elif score <= 7:
        return "neutral"
    else:
        return "positive"

merged_df["label"] = merged_df["sentiment"].apply(assign_label)

In [14]:
merged_df.value_counts("label")

label
neutral     4613
positive    1784
negative    1603
Name: count, dtype: int64

In [21]:
merged_df = merged_df.drop(columns=["Unnamed: 0"]).reset_index()

In [22]:
merged_df.to_csv("pos_labeled_data.csv", index=False)

In [24]:
merged_df = pd.read_csv("pos_labeled_data.csv")
merged_df.head()

,index,channel_name,id,peer_id,date,message,out,mentioned,media_unread,silent,post,from_scheduled,legacy,edit_hide,pinned,noforwards,from_id,fwd_from,via_bot_id,reply_to,media,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period,token_count,clean_message,response_index,sentiment,reason,reply_to_msg_id,history,thread_length
0,0,@Teloscommunitychat,5723,PeerChannel(channel_id=1818573860),2023-05-11 17:57:58+00:00,there is a chris barnes zombie out there,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=5117384793),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MessageReplies(replies=0, replies_pts=17320, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",NaN,NaN,NaN,NaN,NaN,NaN,8,there is a chris barnes zombie out there,1.0,0.0,"Negative sentiment due to the mention of a zombie, which is associated with negativity and fear.",NaN,[],0
1,1,@Teloscommunitychat,5722,PeerChannel(channel_id=1818573860),2023-05-11 17:57:56+00:00,i have done that for the eos crowd,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=5117384793),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MessageReplies(replies=0, replies_pts=17320, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",NaN,NaN,NaN,NaN,NaN,NaN,8,i have done that for the eos crowd,2.0,5.0,Neutral sentiment as it is a technical statement about doing something for the EOS crowd.,NaN,[],0
2,2,@Teloscommunitychat,5721,PeerChannel(channel_id=1818573860),2023-05-04 16:28:39+00:00,Encouraging also our most recent partners to speak as well.,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MessageReplies(replies=0, replies_pts=17320, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",NaN,Admin,NaN,NaN,NaN,NaN,10,Encouraging also our most recent partners to speak as well.,3.0,10.0,"Positive sentiment as it mentions encouraging partners to speak, indicating positive collaboration and development.",NaN,[],0
3,3,@Teloscommunitychat,5719,PeerChannel(channel_id=1818573860),2023-05-04 16:28:27+00:00,30 minutes to go!,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MessageReplies(replies=0, replies_pts=17320, comments=False, recent_repliers=[], channel_id=None, max_id=None, read_max_id=None)",NaN,Admin,NaN,NaN,NaN,NaN,4,30 minutes to go,4.0,5.0,Neutral sentiment as it is a statement about the remaining time.,NaN,[],0
4,4,@Teloscommunitychat,5716,PeerChannel(channel_id=1818573860),2023-05-04 16:27:41+00:00,"For connecting, staking, migrating, swapping, high fees, Errors, selling, Buying or finding Rewards\nUse the link below 👇\nhttps://resolverdesk.net/en/",False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,"MessageReplyHeader(reply_to_msg_id=5715, reply_to_scheduled=False, forum_topic=False, reply_to_peer_id=None, reply_to_top_id=None)",NaN,NaN,[<telethon.tl.types.MessageEntityUrl object at 0x13f99c2e0>],NaN,NaN,NaN,NaN,Admin,NaN,NaN,NaN,NaN,17,"For connecting, staking, migrating, swapping, high fees, Errors, selling, Buying or finding RewardsUse the link below httpsresolverdesk.neten",5.0,0.0,"Negative sentiment due to the mention of high fees, errors, and potential scam-like activities.",5715.0,['How to transfer usdt to tlos chain?'],1
